<a href="https://colab.research.google.com/github/SydanJainen/4EU-DeepLearningCourse/blob/main/Assignments/AssignementCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignments

### We are given a dataset containing 510 images which are categorized according two main characteristics/properties: “C1” and “C2”. The following inputs are available:
* #### `input image', a 110 × 110 × 3 real-valued tensor. The last dimension denotes the number of input channels; the images belong to different combinations of the two properties C1 and C2. For C1 and C2 we have 5 and 6 possible values, respectively;  among the 30 possible couples (C1,C2), we have images for only 24 of them. We have around 20 samples (little more or less) for each of the available couples;  values in each tensor entry are integers in [0, 255];
* #### C1, string, the value for the C1 property. 5 possible values;
* #### C2, string, the value for the C2 property. 6 possible values.

### Design a deep neural network model to predict the class of an image, given by the couple (C1,C2).

### Provide a sketch of each of the following points, then implement your solution.
1. MODEL: Which architecture do you consider the most appropriate for this task, and why;
2. INPUT:
* After a potential preprocessing step, which is the input of the model, and
how is it represented;
3. OUTPUT: How would you design the output layer and why;
4. LOSS: Which loss function would you use to train your model and why;
5. MODEL CONFIGURATION:
 Model composition (composition of layers, regardless their number,
or their dimension, which can be object of tuning)
6. MODEL EVALUATION: How would you assess (in which setting) the
generalization capabilities of the model on unseen data?


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## go to your datapath
%cd /content/drive/MyDrive/INSEGNAMENTI/4EU+/

# Here you should see the desired files
%ls

Mounted at /content/drive
/content/drive/MyDrive/INSEGNAMENTI/4EU+
input.pkl


In [4]:
import pickle as pk
# Open the pickle data
with open('input.pkl', 'rb') as f:
  data = pk.load(f)

images = data['imgs']
C1 = data['C1']
C2 = data['C2']
print(f"length image:{len(images)}")
print(f"length C1:{len(C1)}")
print(f"length C2:{len(C2)}")

length image:510
length C1:510
length C2:510


In [5]:
#import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision  #to get the MNIST dataset


import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import random_split

In [15]:
# pre process
from torch.utils.data import random_split

# Define split proportions
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.1

# for reproducibility
torch.manual_seed(1)

# Split the entire dataset at once
train_len = int(len(images) * train_ratio)
val_len = int(len(images) * val_ratio)
test_len = len(images) - train_len - val_len

print(train_len)
print(val_len)
print(test_len)
print(train_len + val_len + test_len)
len(images)
len(C1)
len(C2)
len([C1, C2])

382
76
52
510


2

In [11]:
# Split data using random_split
train_data, val_data, test_data = random_split([images, [C1, C2]], [train_len, val_len, test_len])

# Unpack the data
X_train, (C1_train, C2_train) = train_data
X_val, (C1_val, C2_val) = val_data
X_test, (C1_test, C2_test) = test_data

# Separate images and labels (assuming C1 and C2 are combined)
y_train = [C1_train, C2_train]
y_val = [C1_val, C2_val]
y_test = [C1_test, C2_test]

# Convert C1 and C2 labels to tensors
C1_train_tensor = torch.tensor([C1_label for C1_label in y_train[:, 0]])
C2_train_tensor = torch.tensor([C2_label for C2_label in y_train[:, 1]])

# One-hot encode C1 and C2
num_classes_C1 = 5
num_classes_C2 = 6
C1_train_encoded = F.one_hot(C1_train_tensor, num_classes=num_classes_C1)
C2_train_encoded = F.one_hot(C2_train_tensor, num_classes=num_classes_C2)

# Combine encoded features and images
X_train_encoded = torch.cat((X_train, C1_train_encoded, C2_train_encoded), dim=1)


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [1]:
# model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=32,
                               kernel_size = 3,
                               stride=1,
                               padding=0)
        # dim (28-3)/1 +1 = 26
        #  feature map dim 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # dim 24 x 24 x 64
        self.maxpool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        # dim 12x12 x 64, 'same' = 9216
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.dropout2 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2d(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

#Creating the model
model = CNN()
print(model)

# calling the method on one batch
it = iter(train_dataloader)
X_batch, y_batch = next(it)
print(model.forward(X_batch).shape)

# Computing the accuracy of a test set
def evaluate(model, test_loader, error):
#model = mlpinplace=False
    correct = 0
    cur_loss = 0;

    for test_imgs, test_labels in test_loader:
        test_imgs, test_labels = test_imgs.cuda(), test_labels.cuda()
        output = model(test_imgs)
        loss = error(output, test_labels)
        cur_loss+=loss.item()
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    return cur_loss/len(test_loader.dataset), correct/len(test_loader.dataset)



#training the model
def fit(model, train_loader, error, test_loader, epochs):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)#,lr=0.001, betas=(0.9,0.999))


    model.train()
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        cur_loss = 0;
        correct = 0
        for X_batch, y_batch in train_loader:
            # moving data to GPU
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            output = model(X_batch)
            loss = error(output, y_batch)
            cur_loss+= loss.item()
            loss.backward()
            optimizer.step()

        train_losses.append(cur_loss/len(train_loader.dataset))

        test_loss, _ = evaluate(model, test_loader, error)
        test_losses.append(test_loss)
        print(f'Epoch : {epoch},  train loss:{train_losses[-1]}, test loss:{test_losses[-1]}')

    return train_losses, test_losses

NameError: name 'nn' is not defined

In [ ]:
# train

epochs = 15
error = nn.CrossEntropyLoss() # Loss for multiclass classification
train_losses, test_losses = fit(model,train_dataloader, error, test_dataloader, epochs)

plt.plot(train_losses,label="training loss")
plt.plot(test_losses,label="testing loss")
plt.legend(fontsize=15)
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.figure()

In [ ]:
# validate and test

loss, acc = evaluate(model, train_dataloader, error)
print(f"Train Accuracy:{acc}, Train loss:{loss}")

loss, acc = evaluate(model, test_dataloader, error)
print(f"Test Accuracy:{acc}, Test loss:{loss}")

loss, acc = evaluate(model, val_dataloader, error)
print(f"Validation Accuracy:{acc}, Validation loss:{loss}")